# crop_square_centered_on_qrcode

This notebook performs the following steps:
* detects QRcode and calculates its centroid
* extracts a square from the image centered on the QRcode

In [6]:
from pyzbar.pyzbar import decode
import cv2
import glob
import os
import papermill
import pandas as pd

In [7]:
# This cell is tagged as "parameters" to enable use of papermill

DATADIR = '/home/aubrey/Desktop/gpepp_nursery'
CROP_WIDTH = 2500

In [8]:
# papermill.inspect_notebook('crop_square_centered_on_qrcode.ipynb')

In [9]:
def crop_square_centered_on_qrcode(img, crop_width):
    code = decode(img)
    rect = code[0].rect
    x_center = rect.left + (rect.width // 2)
    y_center = rect.top + (rect.height // 2)

    half_width = crop_width // 2
    start_row = y_center - half_width
    end_row = y_center + half_width
    start_col = x_center - half_width
    end_col = x_center + half_width

    # If crop_width is even, we add an extra row and column
    if (crop_width % 2):
        end_row += 1
        end_col += 1 
        
    return img[start_row:end_row, start_col:end_col]

In [10]:
# MAIN

df_imagelist = pd.read_csv(f'{DATADIR}/imagelist.csv')
for i, r in df_imagelist.iterrows():
    sn = os.path.basename(r.filename).replace('.png', '')
    original_image_path = f'{DATADIR}/{sn}/original.png'
    cropped_image_path =  f'{DATADIR}/{sn}/cropped.png'
    print(f'Processing{original_image_path}/{sn}')
    img = cv2.imread(original_image_path)
    cropped_img = crop_square_centered_on_qrcode(img, CROP_WIDTH) 
    cv2.imwrite(cropped_image_path, cropped_img)
print('Finished.')    

Processing/home/aubrey/Desktop/gpepp_nursery/31/original.png/31
Processing/home/aubrey/Desktop/gpepp_nursery/26/original.png/26
Processing/home/aubrey/Desktop/gpepp_nursery/28/original.png/28
Processing/home/aubrey/Desktop/gpepp_nursery/27/original.png/27
Processing/home/aubrey/Desktop/gpepp_nursery/38/original.png/38
Processing/home/aubrey/Desktop/gpepp_nursery/35/original.png/35
Processing/home/aubrey/Desktop/gpepp_nursery/33/original.png/33
Processing/home/aubrey/Desktop/gpepp_nursery/32/original.png/32
Processing/home/aubrey/Desktop/gpepp_nursery/30/original.png/30
Processing/home/aubrey/Desktop/gpepp_nursery/25/original.png/25
Processing/home/aubrey/Desktop/gpepp_nursery/29/original.png/29
Finished.
